### Section 0: Loading packages and testing earth engine connection

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import ee
import earthlib as eli
import geemap

In [ ]:
# OPTIONAL: authenticate earth engine api access (not required on every run)
ee.Authenticate()

In [ ]:
# initialize earth engine api access (required on every run)
ee.Initialize()

# test whether the initialization was successful by printing the elevation of Mount Everest
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('Point location elevation (m):', elev)

### Section 1: unmixing one image

In [ ]:
# select the sensor to use
sensor = 'Landsat8'
collection = eli.getCollection(sensor)
bands = eli.getBands(sensor)
scaler = eli.Scale.bySensor(sensor)
cloudMask = eli.CloudMask.bySensor(sensor)
shadeMask = eli.ShadeMask.bySensor(sensor)
brightMask = eli.BrightMask.bySensor(sensor)
BRDF = eli.BRDFCorrect.bySensor(sensor)
NIRv = eli.NIRv.bySensor(sensor)
visBands = ["SR_B6", "SR_B5", "SR_B4"] # Landsat-8
#visBands = ["Nadir_Reflectance_Band6", "Nadir_Reflectance_Band2", "Nadir_Reflectance_Band1"] # MODIS
#visBands = ["B11", "B8", "B4"] # Sentinel-2

# set the dates
startDate = "2021-01-01";
endDate = "2021-09-01";

# get a land mask to use
land = ee.Image("UMD/hansen/global_forest_change_2019_v1_7").select(["datamask"]).eq(1)

# create and filter the image collection
filtered = collection.filterDate(startDate, endDate).map(cloudMask).select(bands)
correction = filtered.map(BRDF)

In [ ]:
# create a composite
sr = filtered.map(scaler).map(shadeMask).map(brightMask).map(eli.CloudMask.Opening).median()
src = correction.map(scaler).median()

# Set visualization parameters.
visParams = {
  'min': 0,
  'max': 0.5,
  'bands': visBands,
}

# Create a folium map object.
Map = geemap.Map(center=[37.5, -118], zoom=6)

# Add the image to the map object.
Map.addLayer(sr, visParams, sensor)
Map.addLayer(src, visParams, f'{sensor} corrected')

# Display the map.
display(Map)

now we'll actually unmix the image using the first spectra from each bundle

In [ ]:
# initialize the spectral bundle selection
eli.Unmix.Initialize(sensor, 10, bands)

# unmix the thing
unmixed = eli.Unmix.SVN(src, shade_normalize=True).updateMask(land)

# map it
visParams = {'min': [0, 0.1, 0], 'max': [0.8, 1, 0.8]}
Map.addLayer(unmixed, visParams, 'unmixed')
display(Map)